# Introduction

In [273]:
import math
import pprint
from collections import deque

import numpy as np
np.seterr(invalid='raise')

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris, load_breast_cancer, load_boston
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

# Setup

In [17]:
def make_labelled(column):
    mean = column.mean()
    for i in range(column.shape[0]):
        column[i] = int(column[i] >= mean) 
    return column

breast = load_breast_cancer()
iris = load_iris()
dataset = iris

X = iris.data
y = dataset.target

#converting iris dataset's continuous features, to discrete features
for i in range(X.shape[1]):
    X[:,i] = make_labelled(X[:,i])

xTrain, xTest, yTrain, yTest = train_test_split(X,y)

# Decision Tree

In [516]:
class Node:
    def __init__(self, df, feature=None, value=None, score=None, children=None):
        #datapoints for current node
        self.df = df
        
        #feature name
        self.feature = feature
        
        #feature value
        self.value = value
        
        #node impurity
        self.score = score
        
        #child nodes (a node for each value of feature)
        #mapping of {value:child node}
        self.children = children
        
        #if node is a leaf node then setting the class label for this leaf node
        if self.children is None:
            try:
                self.label
            except:
                self.label = self.df.y.unique()[0]
                
        
    def __str__(self):
        if self.children is not None:
            return """
                feature: {0}
                score: {1}
                children: {2}
            """.replace('    ','').strip('\n').format(self.feature, self.score, len(self.children))
        else:
            return """
                value: {0}
                score: {1}
                label: {2}
            """.replace('    ','').strip('\n').format(self.value, self.score, self.label)

# DecisionTreeClassifier

In [544]:
class CustomDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None, split='gini_index', feature_names=None, verbose=False):
        self.max_depth = max_depth
        self.split = split
        self.feature_names = feature_names
        self.verbose = verbose
    
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        
        #unique classes and no of features
        self.classes = list(set(y))
        self.xdim = X.shape[1]
        
        #creating dataframe
        df = pd.DataFrame(X)
        if self.feature_names != None:
            df.columns = self.feature_names
        df['y'] = y
        self.df = df
        
        #constructing decision tree
        self._build_decision_tree(self.df)
        
        self.fitted_ = True
        
        return self
    
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            y_preds = []
            for x in X:
                temp = self.root
                
                #traversing our tree until we have reached the leaf node
                while True:
                    if temp.children is None: break
                    else:
                        index = self.feature_names.index(temp.feature)
                        temp = temp.children[x[index]]
                    
                #we are at leaf node therefore setting class label of leaf node as our prediction
                y_pred = temp.label
                y_preds.append(y_pred)
            return np.array(y_preds)
    
    
    def _build_decision_tree(self, df):
        #initializing empty queue
        queue = deque()
        root = None
        
        while True:
            if len(queue) == 0:
                #getting purity score for each feature
                #returns {feature=>score}
                scores = self._scores(df)

                #getting feature to split on (feature with minimum impurity)
                feature = min(scores, key=lambda k: scores[k])
                
                #getting score of selected feature
                score = scores[feature]

                #if node is pure
                if score == 0:
                    #creating root node
                    root = Node(df=df, score=scores[feature])
                    root.label = df.y[0]
                    
                #if all features have same impurity
                #which means dataset cannot divided further
                #therefore selecting class label with highest frequency as leaf node's label
                elif len(np.unique(list(scores.values()))) == 1:
                    #creating root node
                    root = Node(df=df, score=scores[feature])
                    node.label = np.argmax(np.bincount(df.y))
                    
                #else continuing creating building tree by child nodes
                else:
                    #creating root node
                    root = Node(df=df, feature=feature, score=scores[feature])
                    
                    #getting unique values for feature to split on
                    values = df[feature].unique()
                    
                    #setting children of current node
                    #for each value of feature creating a child with its specific df and value
                    children = {}
                    for value in values:
                        child = Node(df=df[df[feature] == value], value=value)
                        children[value] = child

                    #adding children to queue 
                    queue = queue + deque(children.values())

                    #setting child nodes
                    root.children = children
                
                #adding root as instance variable for tree traversal in prediction phase
                self.root = root
                
                #printing node info if verbose is True
                if self.verbose:
                    print(root)
                    print()
            elif len(queue) > 0:
                node = queue.popleft()
                
                #getting impurity score for each feature
                #returns {feature=>score}
                scores = self._scores(node.df)

                #setting impurity score of node (lowest value)
                node.score = scores[feature]
                
                #if node is pure
                if node.score == 0:
                    pass
                
                #if all features have same impurity
                #which means dataset cannot be divided further
                #therefore selecting class label with highest frequency as leaf node's label
                elif len(np.unique(list(scores.values()))) == 1:
                    node.label = np.argmax(np.bincount(node.df.y))
                    
                #else continuing creating building tree by child nodes
                else:
                    #getting feature to split on (feature with minimum impurity)
                    feature = min(scores, key=lambda k: scores[k])

                    #setting feature to split on
                    node.feature = feature

                    #getting unique values for feature to split on
                    values = node.df[feature].unique()
                
                    #setting children of current node
                    #for each value of feature creating a child with its specific df and value
                    children = {}
                    for value in values:
                        child = Node(df=node.df[node.df[feature] == value], value=value)
                        children[value] = child

                    #adding children to queue 
                    queue = queue + deque(children.values())

                    #setting child nodes
                    node.children = children
                    
                #printing node info if verbose is True
                if self.verbose:
                    print(node)
                    print()
            
            if len(queue) == 0:
                break
    
    def _cost(self, groups):
        pass
    
    
    def _gini_index(self, df, feature):
        gini_index = 0

        #getting unique values of current feature
        values = df[feature].unique()

        gindices = [1 for _ in range(len(values))]

        #for each value of feature calculating gini value
        for i,value in enumerate(values):
            for c in self.classes:
                gindices[i] -= (df.loc[df[feature] == value].loc[df.y == c].count().y / df[df[feature] == value].count().y) ** 2

        #calculating gini index
        for i, value in enumerate(values):
            gini_index += gindices[i] * (df[df[feature] == value].count().y / df.count().y)

        return gini_index
    
    
    def _gini_indices(self, df):
        gini_indices = {}
        
        #getting gini index for each feature
        for feature in df.columns[0:-1]:
            gini_indices[feature] = self._gini_index(df, feature)
            
        return gini_indices
        
    
    #@TODO
    def _proportions(self, df):
        total_rows = len(df)
        ps = {}

        for c in self.classes:
            ps[c] = len(df[df.y == c]).count().y / total_rows
        
        return ps
    
    
    def _entropy(self, ps):
        return -(ps * np.log2(ps)).sum()
                               
        
    #@TODO
    def _gain(self, original, children):
        original_ps = self._proportions(original)
        original_entropy = self._entropy(original_ps)
        
        split_entropy = 0
        for child in children:
            child_ps = self._proportions(children.df)
            split_entropy += self._entropy(child_ps)
        split_entropy = split_entropy / len(children)
        
        return original_entropy - split_entropy
    
    
    #@TODO
    def _split_entropy(self, original, children):
        pass
    
    
    #@TODO
    def _gain_ratio(self, df):
        scores = {}
        
        for feature in df.columns:
            groups = df.groupby(by=feature)
            feature_df = groups.get_group(feature)
            feature_dfs.append(feature_df)
            
        return scores
    
    
    def _scores(self, df):
        if self.split == 'gain_ratio':
            scores = self_gain_ratios(df)
        elif self.split == 'gini_index':
            scores = self._gini_indices(df)
        
        return scores
    
    
    #@TODO
    def print_decision_tree(self):
        pass
    
                               
    def _check_params(self, X, y=None):
        if type(self.split) != str:
            raise Exception('type of "split" should be string')
        elif self.split not in ('gini_index', 'gain_ratio'):
            warnings.warn('invalid value of "split", using default("gini_index") value')
            self.split = 'gini_index'

# DecisionTreeClassifier vs CustomDecisionTreeClassifier

In [548]:
skModel = DecisionTreeClassifier().fit(xTrain, yTrain)
custModel = CustomDecisionTreeClassifier(feature_names=dataset.feature_names).fit(xTrain, yTrain)

print(skModel.score(xTest, yTest))
print(custModel.score(xTest, yTest))

0.815789473684
0.815789473684


In [549]:
print(cross_val_score(estimator=skModel, X=xTest, y=yTest))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest))

[ 0.71428571  0.91666667  0.83333333]
[ 0.71428571  0.91666667  0.83333333]


# Exporting & Understanding DecisionTree